In [117]:
# import os
# import sys
# sys.path.append(os.path.abspath('') + '/..')

# from data import ImplicitData

from flurs.data.entity import User, Item, Event
from flurs.recommender import UserKNNRecommender
import pandas as pd
import numpy as np

In [131]:
# importa dataset 'palco playlists'
data = pd.read_csv("../datasets/ml1m_gte5.csv", sep='\t')
user_col = 'UserID'
item_col = 'ItemID'
data.head()

,UserID,ItemID,Timestamp
0,6040,593,956703954
1,6040,2019,956703977
2,6040,213,956704056
3,6040,3111,956704056
4,6040,912,956704191


In [132]:
data2=data.copy()

In [134]:
usermap = pd.Series(pd.unique( data[user_col] )).reset_index().set_index(0).to_dict()['index']
itemmap = pd.Series(pd.unique( data[item_col] )).reset_index().set_index(0).to_dict()['index']
# 3m 24.7s
# def mapfun(x, unique=pd.unique( data[item_col] )):
#     return np.where( unique == x )[0][0]
# remaping user/item names so that they are ordered from 0 to number of users/items
# Flurs need user/item IDs to be increasing order integers
# data2[item_col]=data2[item_col].map(mapfun)
# data2[user_col]=data2[user_col].map( lambda x: mapfun( x, unique=pd.unique( data[user_col] ) ) )

data2[user_col]=data2[user_col].map(usermap)
data2[item_col]=data2[item_col].map(itemmap)

data2 = data2.iloc[:, :2].values

In [70]:
# Initialize model
knn_RS = UserKNNRecommender(k=5)
knn_RS.initialize()

In [71]:
# For each interaction - register user, register item, register event
for u, i in data2[:1000]:
    # print(u,i)
    user = User(u)
    knn_RS.register(user)

    item = Item(i)
    knn_RS.register(item)

    event = Event(user, item)
    knn_RS.update(event)

In [25]:
knn_RS.users[0]['known_items']

set()

In [75]:
for i in knn_RS.users[0]['known_items']:
    print(i)

In [74]:
knn_RS.recommend(User(0), np.unique( data2[:1000, 1] ) ) # np.unique( data2[:, 1] ).reshape(1,-1)
# => (sorted candidates, scores)

(array([516, 192, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
        173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
        186, 187, 188, 189, 190, 161, 160, 159, 143, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 158, 145, 146,
        147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 191, 193,
        257, 194, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
        251, 252, 253, 254, 255, 226, 225, 224, 208, 195, 196, 197, 198,
        199, 200, 201, 202, 203, 204, 205, 206, 207, 209, 223, 210, 211,
        212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 129, 128,
        127, 126,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
         44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  32,  31,  30,  14,   1,   2,   3,   4,
          5,   6,   7,   8,   9,  10,  11,  12,  13

In [136]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')
from data import ImplicitData, getBucketsHoldouts
from plot_utils import lineplot_recallxholdout, recall_heatmap
from dataset_evaluation_utils import *
from recommenders_implicit import ISGD, RAISGD, RSISGD  # ISGD framework, BISGD,
from eval_implicit import EvaluateHoldouts, EvaluateAndStore # EvaluateAndStore para guardar estados do modelo e holdouts, a avaliação prequencial de ratings implicitos é opcional, , EvalHoldout

from datetime import datetime
import joblib
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

In [137]:
# importa dataset 'books playlists'
data = pd.read_csv('output/amazonbooks_dump/2nd_sampled_amazon_books.csv')
dataset_name = 'Amazon_Books'
user_col = 'user_id'
item_col = 'item_id'
data.sort_values(by='timestamp', inplace=True)

In [150]:
usermap = pd.Series(pd.unique( data[user_col] )).reset_index().set_index(0).to_dict()['index']
itemmap = pd.Series(pd.unique( data[item_col] )).reset_index().set_index(0).to_dict()['index']

In [86]:
buckets = joblib.load('output/amazonbooks_dump/2nd_sample_buckets.joblib')
holdouts = joblib.load('output/amazonbooks_dump/2nd_sample_holdouts.joblib')

In [151]:
b0 = buckets[0]
u,i = b0.GetTuple(0)
u, b0.GetUserItems(u, internal=False)

In [152]:
import copy
model_checkpoints = []
results_matrix = np.zeros( shape=( len( holdouts ), len( holdouts ) ) )
# Initialize model
knn_RS = UserKNNRecommender(k=5)
knn_RS.initialize()
cold_start_buckets = 1
for b in buckets:
    for int in range(b.size):
        u,i = b.GetTuple(int)
        u_flurs, i_flurs = usermap[u], itemmap[i]
        # For each interaction - register user, register item, register event
        user = User(u)
        knn_RS.register(user)
        item = Item(i)
        knn_RS.register(item)
        event = Event(user, item)
        knn_RS.update(event)
    if b >= cold_start_buckets:
        model_cp = copy.deepcopy(knn_RS)
        model_checkpoints.append(model_cp)
    else:
        pass    
    # after learning a bucket, evaluate model over every holdout
    user_items = [ itemmap[i] for i in b.GetUserItems(u, internal=False) ]
    for h in holdouts:

    



0 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 14
2 15
3 16
2 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
1 30
4 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
5 44
6 45
6 46
6 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
4 64
4 65
4 66
7 67
7 68
7 69
7 70
4 71
4 72
4 73
4 74
4 75
4 76
4 77
4 78
4 79
4 80
2 81
8 82
8 83
8 84
8 85
8 86
8 87
8 88
8 89
8 90
9 91
9 92
9 93
2 94
2 95
7 96
7 97
4 98
7 99
4 100
4 101
4 102
4 103
4 104
4 105
4 106
4 107
4 108
4 109
4 110
4 111
4 112
4 113
1 114
4 115
7 116
7 117
7 118
7 119
7 120
7 121
7 122
7 123
7 124
7 125
7 126
7 127
7 128
7 129
4 130
8 131
1 132
1 133
10 134
10 135
11 136
12 137
11 138
11 139
13 140
13 141
13 142
13 143
13 144
13 145
12 146
12 147
12 148
12 149
12 150
3 151
3 152
14 153
2 154
15 155
15 156
15 157
15 158
15 159
15 160
15 161
15 162
10 163
15 164
10 165
10 166
16 167
16 168
16 169
16 170
16 171
12 172
16 173
16 174
16 175
16 176
16 177
1

KeyboardInterrupt: 